# La crate web-sys et le DOM

---

> La crate web-sys vient en complément de wasm-bindgen et elle permet d'utiliser directement les API web depuis rust. Elle permet donc premièrement de pouvoir accéder aux éléments du DOM, d'en ajouter, supprimer etc. mais elle permet aussi d'utiliser des fonctionnalités plus avancées comme WebGL ou les websockets par exemple.

fichier **cargo.toml** :
```
[dependencies.web-sys]
version = "0.3.63"
features = [
  'Document',
  'Element',
  'HtmlElement',
  'Node',
  'Window',                                                                                                     ... Some more features ...                                                             
] 
```
**Notez que pour chaque fonctionnalité que l'on souhaite utiliser, on doit rajouter une feature, ainsi pour utiliser un canva 2D, il est nécessaire de rajouter `HtmlCanvasElement` et `CanvasRenderingContext2d`**.

The `wasm-bindgen` Guide introduit quelques notions sur web-sys chapitre **3**, et beaucoup d'exemple qui y sont présentés utilisent web-sys. Vous pouvez trouver la documentation complète de la crate web-sys a cette adresse : [https://rustwasm.github.io/wasm-bindgen/api/web_sys/](https://rustwasm.github.io/wasm-bindgen/api/web_sys/).

## La fonction window

- La crate web_sys ne définit qu'une seule fonction dans son namespace principal, c'est la fonction `window` :

> ```
>Function web_sys::window
>
>pub fn window() -> Option<Window>
>
>Getter for the Window object
>
>MDN Documentation
>
>This API requires the following crate features to be activated: Window
>```

`let window = web_sys::window().expect("no global window exists");`
    
*Cette fonction retourne l'objet Window du navigateur. Notons aussi qu'il faut déjà activer la feature Window pour que cela fonctionne.*

À partir de window, on descend par exemple vers l'élément qui nous intéresse comme on le ferait en Javascript.

>```
>impl Window
>
>pub fn document(&self) -> Option<Document>
>
>Getter for the document field of this object.
>
>MDN Documentation
>
>This API requires the following crate features to be activated: Document, Window
>
>```

`let document = window.document().expect("should have a document on window");`

>```
>impl Document
>
>pub fn body(&self) -> Option<HtmlElement>
>
>Getter for the body field of this object.
>
>MDN Documentation
>
>This API requires the following crate features to be activated: Document, HtmlElement
>
>```

`let body = document.body().expect("document should have a body");`
    
*Déjà, trois features sont utilisées. `Document`, `HtmlElement` et `Window`*

## La manipulation du DOM

> Puisque le but de web-sys est de donner accès aux API web, elle donne bien entendu accès à l'API Document, ainsi, il devient possible de faire les mêmes choses que l'on ferait en JS, par exemple :

`let ma_div = document.create_element("div").unwrap();`

`let ma_seconde_div = document.get_element_by_id("terminal").unwrap();`

*Ces fonctions retournent un **Element** (A ajouter aux features donc...)*

### Implémentations de Deref pour HtmlElement.

D'après la documentation de web-sys, la méthode `append_child` est implémentée pour une structure de type `Node` :

>```
>impl Node
>
>pub fn append_child(&self, node: &Node) -> Result<Node, JsValue>
>
>The appendChild() method.
>
>MDN Documentation
>
>This API requires the following crate features to be activated: Node
>```

Seulement, les types `HtmlElement` et `Element` implémentent le trait `Deref<Target = Node>`, c'est ainsi qu'il est possible d'utiliser la fonction `append_child` sur un `HtmlElement` comme ceci :

`let new_node = mon_html_element.append_child(&un_element).unwrap();`

*Illustration d'implementation du trait Deref :*

In [2]:
{
    #[derive(Debug)]
    struct InnerElement {
        value: usize,
    }

    impl InnerElement {
        fn inner_methode(&self) -> usize {
            self.value
        }
    }

    #[derive(Debug)]
    struct Element {
        inner: InnerElement,
    }

    impl std::ops::Deref for Element {
        type Target = InnerElement;

        fn deref(&self) -> &Self::Target {
            &self.inner
        }
    }

    let b = Element { inner: InnerElement { value: 42 }};
    println!("Hello, world! {:?}", b.inner_methode());
}


Hello, world! 42


()

## Exercice

Le prototype de la fonction open de Terminal est :
```                              
    fn open(this: &Terminal, element: &web_sys::HtmlElement); 
```

>```
>open
>
>▸ open(parent: HTMLElement): void
>
>Defined in xterm.d.ts:879
>
>Parameters:
>Name 	Type 	Description
>parent 	HTMLElement 	The element to create the terminal within. This element must be visible (have >dimensions) when open is called as several DOM- based measurements need to be performed when this function is >called.
>
>Returns: void
>```

**Element & Node implémentent tous les deux le `trait JsCast` qui permet le cast entre les types JS.**

```
EventTarget
│
└── Node
    │
    └── Element
        │
        ├── HTMLElement
        │   │
        │   ├── HTMLDivElement
        │   ├── HTMLParagraphElement
        │   ├── HTMLSpanElement
        │   ├── ...
        │
        ├── SVGElement
        ├── ...
```
**INITIALISEZ UNE INSTANCE DE LA CLASSE TERMINAL ET UTILISEZ LES MÉTHODES OPEN ET WRITE DEPUIS LE FICHIER WASM.**